### Loading the libraries

In [1]:
import os
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


### Data Loading and exploration

In [2]:
fpath = "dataset/images/Images/"
random_seed = 42

categories = os.listdir(fpath)
categories = categories[:20]
print("List of Categories = ", categories, "\n\nNo. of categories = ", len(categories))

List of Categories =  ['n02088364-beagle', 'n02099601-golden_retriever', 'n02099712-Labrador_retriever', 'n02102318-cocker_spaniel', 'n02106550-Rottweiler', 'n02106662-German_shepherd', 'n02107142-Doberman', 'n02108089-boxer', 'n02108422-bull_mastiff', 'n02108551-Tibetan_mastiff', 'n02109047-Great_Dane', 'n02110958-pug', 'n02112018-Pomeranian'] 

No. of categories =  13


In [3]:
def load_images_and_labels(categories):
    img_lst = []
    labels = []
    for index, category in enumerate(categories): 
        for image_name in os.listdir(fpath + "/" + category):
            img = cv2.imread(fpath + "/" + category + "/" + image_name)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # converting colorspace from BGR to RGB
            
            img_array = Image.fromarray(img, 'RGB')
            #resize image to 227 x 227 because the input image resolution for AlexNet is 227 x 227
            
            resized_img = img_array.resize((227, 227))
            
            img_lst.append(np.array(resized_img))
            
            labels.append(index)
    return img_lst, labels

images, labels = load_images_and_labels(categories)
print("No. of images loaded = ",len(images),"\nNo. of labels loaded = ",len(labels))
print(type(images),type(labels))

No. of images loaded =  2163 
No. of labels loaded =  2163
<class 'list'> <class 'list'>


In [4]:
images = np.array(images)
labels = np.array(labels)

print("Images shape = ",images.shape,"\nLabels shape = ",labels.shape)
print(type(images),type(labels))

Images shape =  (2163, 227, 227, 3) 
Labels shape =  (2163,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


### Prepare data for training the CNN model

For training the CNN model we have to shuffle all the data that is loaded in images, labels list.

In [5]:
# step 1 in data shuffling

# get equally spaced numbers in a given range
n = np.arange(images.shape[0])
print(" 'n' values before shuffling = ", n)

# shuffle all the equally spaced values in list 'n'
np.random.seed(random_seed)
np.random.shuffle(n)
print("\n'n' values after shuffling = ", n)

 'n' values before shuffling =  [   0    1    2 ... 2160 2161 2162]

'n' values after shuffling =  [ 786 1542 1270 ... 1130 1294  860]


In [6]:
# step 2 in data shuffling

# shuffle images and corresponding labels data in both the lists
images = images[n]
labels = labels[n]


In [7]:
print(images.dtype, labels.dtype)

uint8 int32


Data Normalization

In [8]:
images = images.astype(np.float32)
labels = labels.astype(np.int32)

images  = images / 255

print("Images shape after normalization = ", images.shape)

Images shape after normalization =  (2163, 227, 227, 3)


Splitting the data

In [9]:
x_train, x_test, y_train, y_test = train_test_split(images, labels,
                                                    test_size = 0.2, random_state = random_seed)

print("x_train shape = ",x_train.shape)
print("y_train shape = ",y_train.shape)
print("\nx_test shape = ",x_test.shape)
print("y_test shape = ",y_test.shape)

x_train shape =  (1730, 227, 227, 3)
y_train shape =  (1730,)

x_test shape =  (433, 227, 227, 3)
y_test shape =  (433,)


### Define AlexNet CNN model

In [30]:
model=Sequential()

#1 conv layer
model.add(Conv2D(filters=96,kernel_size=(11,11),strides=(4,4),
                 padding="valid",activation="relu",input_shape=(227,227,3)))

#1 max pool layer
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

model.add(BatchNormalization())

#2 conv layer
model.add(Conv2D(filters=256,kernel_size=(5,5),strides=(1,1),
                 padding="valid",activation="relu"))

#2 max pool layer
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

model.add(BatchNormalization())

#3 conv layer
model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),
                 padding="valid",activation="relu"))

#4 conv layer
model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),
                 padding="valid",activation="relu"))

#5 conv layer
model.add(Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),
                 padding="valid",activation="relu"))

#3 max pool layer
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

model.add(BatchNormalization())


model.add(Flatten())

#1 dense layer
model.add(Dense(4096,input_shape=(227,227,3),activation="relu"))

model.add(Dropout(0.4))

model.add(BatchNormalization())

#2 dense layer
model.add(Dense(4096,activation="relu"))

model.add(Dropout(0.4))

model.add(BatchNormalization())

#3 dense layer
model.add(Dense(1000,activation="relu"))

model.add(Dropout(0.4))

model.add(BatchNormalization())

#output layer
model.add(Dense(20,activation="softmax"))

Compile the cnn model

In [31]:
model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy",
             metrics = ["accuracy"])

### Train the model

In [32]:
model.fit(x_train, y_train, epochs = 50, batch_size = 20)

Epoch 1/50
1730/1730 [==============================] - 11s 6ms/step - loss: 3.5635 - accuracy: 0.0931
Epoch 2/50
1730/1730 [==============================] - 9s 5ms/step - loss: 3.0887 - accuracy: 0.1283
Epoch 3/50
1730/1730 [==============================] - 9s 5ms/step - loss: 2.9499 - accuracy: 0.1289
Epoch 4/50
1730/1730 [==============================] - 9s 5ms/step - loss: 2.8453 - accuracy: 0.1439
Epoch 5/50
1730/1730 [==============================] - 9s 5ms/step - loss: 2.7680 - accuracy: 0.1387
Epoch 6/50
1730/1730 [==============================] - 9s 5ms/step - loss: 2.7411 - accuracy: 0.1549
Epoch 7/50
1730/1730 [==============================] - 9s 5ms/step - loss: 2.6873 - accuracy: 0.1549
Epoch 8/50
1730/1730 [==============================] - 9s 5ms/step - loss: 2.6140 - accuracy: 0.1595
Epoch 9/50
1730/1730 [==============================] - 9s 5ms/step - loss: 2.5309 - accuracy: 0.1855
Epoch 10/50
1730/1730 [==============================] - 9s 5ms/step - loss: 2.54

In [33]:
predictions = model.predict(x_test)

In [34]:
results = model.evaluate(x_test, y_test, batch_size = 50)
print(results)

433/433 [==============================] - 1s 2ms/step
[6.401017335490872, 0.26096996665000916]


In [17]:
results

[5.273834789047065, 0.21709007024765015]